<table style="width:100%;">
    <tr style="display:none">
        <td></td>
        <td></td>
    </tr>
    <tr style="height:3em">
        <td style="width:60%;font: bold 2.5em 'Fira Sans', serif;text-align:center">SCRIPT DE REMPLISSAGE DE BASE   - PIB <br>
        Durée d'exécution :  60 sec<br>
        <br>
    </tr>
</table>

<center>



## Il est important d'avoir les librairies suivantes : 
- **Pandas** afin de traiter le fichier csv
- **Pyodbc** afin de faire du sql et surtout pouvoir ce connecter à la base 
- **Pycountry** convert pour avoir le nom du continent d'un pays
- **Date time** afin de convertir une valeur en date
- **Time** d'avoir le temps d'exécution

In [1]:
import pandas as pd
import pyodbc
import pycountry_convert as pc
from datetime import datetime
import time

## Détails de connexion :
Afin de pouvoir vous connecter à la base de données  il va falloir saisir les informations suivantes : 
- Le nom du serveur
- Le nom de la base de données
- Le nom d'utilisateur
- Le mot de passe

In [2]:
server = 'INFO-MSSQL-ETD'
database = 'SAE_TEAM55'
username = 'etd05'
password = 'fqnsbtc4'


## Le chemin du fichier csv


In [3]:
fichiers = [
    'Z:\\Documents\\saeTruc\\processed_gdp_worldbankTranslated.csv',
]

Afin de pouvoir avoir la région d'un pays on va stocker dans un tableau associatif en tant que clé l'indicatif du continent et en valeur le nom du continent.
Il y a également un tableau **cont** qui va stocker les noms des continents. Cela va permettre : 
- Si un nom de pays est un nom de continent dans le csv alors, la région et le continent seront identiques (le nom du continent).



In [4]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe',
    'WW': 'World'
}
cont = ['North America','South America','Asia','Australia','Africa','Europe','World','Caribbean',
        'demographic','Euro','conflict','poor','income','IBRD','IDA','countries','OECD','small','Small',
        'Bahamas','Channel','Dem','Rep','Curacao','Hong']

voici donc les fameuses méthodes qui permettent d'avoir les bons nom de continents et de pays.
- `get_Region(country_name)` : 
Permet de transformer le nom d'un pays en son continent à partir de l'indicatif du pays et le retourner grâce à la bibliothèque `pycountry_convert`  exemple : 
- France  **====>** FRA **=======>** EUROPE
- `get_region_and_country(filename)` : 
cette méthode permet de regarder si le nom initial est un continent ou un pays. Dans le cas d'un continent le nom du pays et le nom du continent sont tout les deux le nom du continent. exemple :
- AFRICA **=====>** region_name = AFRICA  country_name = AFRICA**
Dans le cas d'un nom de pays on va appeler la méthode `get_Region(country_name)` afin d'avoir le continent. exemple : 
- QATAR **=======>** region_name = QATAR  country_name = QATAR **====>** region_name = ASIA  country_name = QATAR
la méthode retourne donc le nom du continent ainsi que le pays


In [5]:
def get_Region(country_name):
    try:
        country_code = pc.country_name_to_country_alpha2(country_name, cn_name_format="default")
        continent_name = pc.country_alpha2_to_continent_code(country_code)
        return continents[continent_name]
    except KeyError as e:
        print(f"KeyError encountered: {str(e)}")
        return country_name
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return country_name

def get_region_and_country(filename):
    region_name = filename
    country_name = filename
    #cas de mauvaise orthographie ou de nom tronqué
    if region_name == "Cote d'Ivoire":
        country_name = 'Ivory Coast'
    if region_name == 'Gambia, The':
        country_name = 'Gambia'
    #parcours du tableau associatif de continents pour regarder si
    for continent in cont:
        if continent in region_name:
            return region_name, country_name
    else:
        region_name = get_Region(country_name)
        return region_name, country_name


Il est important de bien vérifier si la date est cohérente, 
une simple méthode peut le faire :

In [6]:
def is_valid_date(year):
    try:
        datetime.strptime(f"{year}-01-01", "%Y-%m-%d")
        return True
    except ValueError:
        return False

## Insertion de données dans la base
C'est le coeur du code, la méthode `process_row(year, region, value, filename, conn)` va insérer les données en sql sur notre base de données.
voici les étapes pour bien insérer des données : 
- **Il faut insérer la region**
- **Insérer le pays et lui assigner la région via l'id de cette dernière**
- **Insérer la date**
- **Insérer la donnée et la lié au Pays, la région et la date puis lui assigner une valeur et une unité**
il faut bien évidemment  vérifier avant de faire une requête si la valeur à insérer n'est pas null.

In [7]:
def process_row(year, region, value, filename, conn):
    if not is_valid_date(year):
        print('=================================')
        print(value)
        print(f"Invalid date for year: {year}")
        return

    region_name, country_name = get_region_and_country(region)
    unit = 'DOLLARS'

    with conn.cursor() as cursor:
        try:
            #Insertion de la région
            if region_name:
                cursor.execute("""
                    IF NOT EXISTS (SELECT 1 FROM T_REGION_REG WHERE REG_NOM = ?)
                    BEGIN
                        INSERT INTO T_REGION_REG (REG_NOM) VALUES (?)
                    END
                """, region_name, region_name)
                conn.commit()
                #Récupération de l'id de la région
                cursor.execute("SELECT REG_ID FROM T_REGION_REG WHERE REG_NOM = ?", region_name)
                row = cursor.fetchone()
                if row:
                    region_id = row[0]
                else:
                    region_id = None
                    print("L'id de la région est vide")
            # Insertion du Pays
            if country_name:
                cursor.execute("""
                    IF NOT EXISTS (SELECT 1 FROM T_PAYS_PYS WHERE PYS_NOM = ?)
                    BEGIN
                        INSERT INTO T_PAYS_PYS (REG_ID, PYS_NOM) VALUES (?, ?)
                    END
                """, country_name, region_id, country_name)
                conn.commit()
            #Récupération de l'id du pays
                cursor.execute("SELECT PYS_ID FROM T_PAYS_PYS WHERE PYS_NOM = ?", country_name)
                row = cursor.fetchone()
                if row:
                    country_id = row[0]
                else:
                    country_id = None
            else:
                country_id = None
            #Insertion de la date
            if country_id:
                cursor.execute("""
                    IF NOT EXISTS (SELECT 1 FROM T_DATE_DAT WHERE DAT_DATE = ?)
                    BEGIN
                        INSERT INTO T_DATE_DAT (DAT_DATE) VALUES (?)
                    END
                """, f"{year}-01-01", f"{year}-01-01")
                conn.commit()
            #Récupération de l'id de la date
                cursor.execute("SELECT DAT_ID FROM T_DATE_DAT WHERE DAT_DATE = ?", f"{year}-01-01")
                row = cursor.fetchone()
                if row:
                    date_id = row[0]
                else:
                    date_id = None
            #Insertion de la donnée (id du pays,id de la date, valeur, label, unitée)
                if date_id:
                    cursor.execute("""
                        INSERT INTO T_DATA_DTA (PYS_ID, DAT_ID, DTA_VALEUR, DTA_NOM, DTA_UNITE)
                        VALUES (?, ?, ?, ?, ?)
                    """, country_id, date_id, value, "GPD", unit)
                    conn.commit()
        #Gestion de l'exception
        except pyodbc.Error as e:
            print(f"Erreur lors de l'insertion des données : {e}")

## Phase de lecture du fichier csv
avant d'insérer les méthodes il faut bien évidemment lire le fichier csv. Grâce à Pandas on peut parcourir chaque ligne du csv et les découper.

In [8]:
def process_file(filename, conn):
    df = pd.read_csv(filename, delimiter=',', names=['Code', 'Year', 'country_name', 'Value'], skiprows=1)
    
    for index, row in df.iterrows():
        if ( row['Value'] != 0 and row['Year']> 1990  ):
            process_row(row['Year'], row['country_name'], row['Value'], filename, conn)

## Phase de connection et de démarrage
C'est un peu comme le main, on va essayer de se connecter à la base de données grâce à toutes les informations saisies. Et démarrer la méthode de parcours du csv.

In [ ]:
try:
    #Essai de connection à la base
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connexion réussie à la base de données")
except pyodbc.Error as e:
    print("Erreur de connexion : ", e)
    exit()
start_time = time.time()
# Traitement des fichiers
for fichier in fichiers:
    process_file(fichier, conn)

#Fin
End_time = time.time()
print('finished')
print(f"Temps total utilisé : {End_time - start_time:.2f} secondes")
conn.close()